## 전처리

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd "/content/drive/MyDrive/dataset"

/content/drive/MyDrive/dataset


In [ ]:
import pandas as pd
df = pd.read_csv("./review_tags.csv", encoding="UTF-8", index_col=False)

df = df[df['brand_code'] == "topten10mall.com"]

df.head()

,brand_code,id,sentiment,review_id,review_tag_type_id,keywords,message,product_id,product_name,created_at
0,topten10mall.com,71398,1,838412,28,"0,212",한달 입어본 후 리뷰\n사이즈가 34보다 크게 나와서\n허리끈 졸라 매지 않으면 밖...,51686,BALANCE 반바지,2021-06-10 23:06:08 UTC
1,topten10mall.com,71400,1,838412,18,"197,311",한달 입어본 후 리뷰\n사이즈가 34보다 크게 나와서\n허리끈 졸라 매지 않으면 밖...,51686,BALANCE 반바지,2021-06-10 23:06:08 UTC
2,topten10mall.com,71401,0,838412,25,"0,212",한달 입어본 후 리뷰\n사이즈가 34보다 크게 나와서\n허리끈 졸라 매지 않으면 밖...,51686,BALANCE 반바지,2021-06-10 23:06:08 UTC
3,topten10mall.com,71397,2,838412,30,"0,212",한달 입어본 후 리뷰\n사이즈가 34보다 크게 나와서\n허리끈 졸라 매지 않으면 밖...,51686,BALANCE 반바지,2021-06-10 23:06:08 UTC
4,topten10mall.com,165113,1,865458,23,"157,208],""^#2"":[249,292",병원에서 햇빛알레르기가 엄청 심하다고 해서 불볕 더위에 긴팔을 찾아다니고 있던 중 ...,31681,여성) 리넨 튜닉 긴팔 셔츠,2021-07-02 01:26:07 UTC


In [ ]:
def get_training_corpus():
  return (
    df[i : i + 1000]["message"]
    for i in range(0, len(df), 1000)
  )

training_corpus = get_training_corpus()

In [ ]:
training_corpus

<generator object get_training_corpus.<locals>.<genexpr> at 0x7f4e73c23c30>

## 지도
konlpy 토크나이저

In [ ]:
!apt-get install g++ openjdk-8-jdk python3-dev python3-pip curl
!pip install konlpy
!apt-get install curl git
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
g++ is already the newest version (4:11.2.0-1ubuntu1).
g++ set to manually installed.
curl is already the newest version (7.81.0-1ubuntu1.14).
python3-dev is already the newest version (3.10.6-1~22.04).
python3-dev set to manually installed.
The following additional packages will be installed:
  fonts-dejavu-core fonts-dejavu-extra libatk-wrapper-java libatk-wrapper-java-jni libfontenc1
  libgail-common libgail18 libgtk2.0-0 libgtk2.0-bin libgtk2.0-common libice-dev librsvg2-common
  libsm-dev libxkbfile1 libxt-dev libxtst6 libxxf86dga1 openjdk-8-jdk-headless openjdk-8-jre
  openjdk-8-jre-headless python3-setuptools python3-wheel x11-utils
Suggested packages:
  gvfs libice-doc libsm-doc libxt-doc openjdk-8-demo openjdk-8-source visualvm libnss-mdns
  fonts-nanum fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei fonts-wqy-zenhei
  fonts-indic python-setuptools-doc mesa-utils
The f

In [ ]:
example = '''옷은 이쁘지만 구김이 너무 잘 생겨요'''

In [ ]:
from konlpy.tag import Okt, Mecab, Kkma

okt = Okt()
kkma = Kkma()
# mecab = Mecab()

morphs_okt = okt.morphs(example)
morphs_kkma = kkma.morphs(example)
# morphs_mecab = mecab.morphs(example)

print("okt형태소:",morphs_okt)
print("kkma형태소:",morphs_kkma)
# print("mecab형태소:",morphs_mecab)

okt형태소: ['옷', '은', '이쁘지만', '구김', '이', '너무', '잘', '생겨요']
kkma형태소: ['옷', '은', '이쁘', '지만', '구김', '이', '너무', '잘', '생기', '어요']


## 비지도
sentencepiece

In [ ]:
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 10.0 MB/s eta 0:00:00


In [ ]:
cd /content

/content


In [ ]:
with open('./corpus.txt', 'w', encoding='utf8') as f:
    f.write('\n'.join(df['message']))

In [ ]:
from sentencepiece import SentencePieceTrainer

SentencePieceTrainer.Train(
  input="./corpus.txt",
  model_prefix="petition_bpe",
  vocab_size=25000,
  model_type = "bpe"
)

In [ ]:
from sentencepiece import SentencePieceProcessor

tokenizer = SentencePieceProcessor()
tokenizer.load("./petition_bpe.model")

tokenized_sentence = tokenizer.encode_as_pieces(example)
print("단일문장 토큰화:" ,tokenized_sentence)

encoded_sentence = tokenizer.encode_as_ids(example)
print("단일문장 인코딩:" ,encoded_sentence)

decode_ids = tokenizer.decode_ids(example)
print("단일문장 디코딩:" ,decode_ids)


단일문장 토큰화: ['▁옷은', '▁이쁘지만', '▁구김이', '▁너무', '▁잘', '▁생겨요']
단일문장 인코딩: [587, 14581, 1105, 29, 10, 10516]
단일문장 디코딩: 옷은 이쁘지만 구김이 너무 잘 생겨요


word piece

In [ ]:
!pip install tokenizers

In [ ]:
from tokenizers import Tokenizer
from tokenizers.models import WordPiece
from tokenizers.normalizers import Sequence, NFD, Lowercase
from tokenizers.pre_tokenizers import Whitespace

tokenizer = Tokenizer(WordPiece())
tokenizer.normalizer = Sequence([NFD(), Lowercase()])
tokenizer.pre_tokenizer = Whitespace()

tokenizer.train(["./corpus.txt"])
tokenizer.save("./petition_wordpiece.json")

In [ ]:
from tokenizers import Tokenizer
from tokenizers.decoders import WordPiece as WordPieceDecoder

tokenizer = Tokenizer.from_file("./petition_wordpiece.json")
tokenizer.decoder = WordPieceDecoder()

encoded_sentence = tokenizer.encode(example)
# encoded_sentences = tokenizer.encode_batch(sentences)

print("인코딩 형식:", type(encoded_sentence))

print("단일문장 토큰화:",encoded_sentence.tokens)
# print("여러문장 토큰화:",[enc.tokens for enc in encoded_sentences])

print("단일문장 수 인코딩:",encoded_sentence.ids)
# print("여러문장 수 인코딩:",[enc.ids for enc in encoded_sentences])

print("점수 인코딩에서 문장 변환:", tokenizer.decode(encoded_sentence.ids))

인코딩 형식: <class 'tokenizers.Encoding'>
단일문장 토큰화: ['옷은', '이쁘지만', '구김이', '너무', '잘', '생겨요']
단일문장 수 인코딩: [2129, 17020, 2710, 1358, 1307, 12687]
점수 인코딩에서 문장 변환: 옷은 이쁘지만 구김이 너무 잘 생겨요


## 토크나이저 전처리
kogpt2

In [ ]:
from transformers import AutoTokenizer

model = "skt/kogpt2-base-v2"
old_tokenizer = AutoTokenizer.from_pretrained(model)


config.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

In [ ]:
tokens = old_tokenizer.tokenize(example)
tokens

['▁옷은', '▁이', '쁘', '지만', '▁구', '김', '이', '▁너무', '▁잘', '▁생겨', '요']

In [ ]:
tokenizer = old_tokenizer.train_new_from_iterator(training_corpus, 52000)


In [ ]:
tokens = tokenizer.tokenize(example)
tokens


['▁옷은', '▁이쁘지만', '▁구김이', '▁너무', '▁잘', '▁생겨요']

In [ ]:
print(len(tokens))
print(len(old_tokenizer.tokenize(example)))


6
11
